# Assembly Tree Generation and Evaluation

## Formulation
 - State Space S = {Every Possible Subassembly}
 - Action Space A = {Every Connection that can currently be removed}
 - P(s'|s,a) = Feasability Checks
 - R(s,a) = Cost of removing a certain + the 
 - Graph Assumption for modelling the 

## Notes
 - Next Steps:
     - Randomly generate rewards and show pathing possible
     - Rewards as a distance cost


In [1]:
import sys, random, math, time
import pprint as pp
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
from networkx.drawing.nx_pydot import graphviz_layout

# HELPER FUNCTIONS

In [2]:
# HTML VERSION Drawing
def plotVis(G):
    if(len(G.nodes[1]) > 1):
        newLabs = {}
        for i in G.nodes:
            newLabs[i] = G.nodes[i]['value'].getNCL()
            del G.nodes[i]['value']
        nx.relabel_nodes(G, newLabs, copy=True)
    
    net = Network(notebook=True)
    net.from_nx(G)
    name = str("NumParts=" + str(numParts) + " and NumConstraints=" + str(len(seqConstraint)) + ".html")
    net.show(name)

    
def hierarchy_pos(G, root=None, width=20., vert_gap = 0.4, vert_loc = 0, xcenter = 0.5):  
    # From Joel's answer at https://stackoverflow.com/a/29597209/2966723.  
    def _hierarchy_pos(G, root, width=1., vert_gap = 0.2, vert_loc = 0, xcenter = 0.5, pos = None, parent = None):   
        if pos is None:
            pos = {root:(xcenter,vert_loc)}
        else:
            pos[root] = (xcenter, vert_loc)
        children = list(G.neighbors(root))
        if not isinstance(G, nx.DiGraph) and parent is not None:
            children.remove(parent)  
        if len(children)!=0:
            dx = width/len(children) 
            nextx = xcenter - width/2 - dx/2
            for child in children:
                nextx += dx
                pos = _hierarchy_pos(G,child, width = dx, vert_gap = vert_gap, 
                                    vert_loc = vert_loc-vert_gap, xcenter=nextx,
                                    pos=pos, parent = root)
        return pos          
    return _hierarchy_pos(G, root, width, vert_gap, vert_loc, xcenter)


# Generating Subassemblies in a recursive manner
def recurGen(p, subA, G):
    currCon = list(subA.H.edges())
    if(len(currCon) > 0):
        i = G.number_of_nodes() + 1
        for c in currCon:
            if(random.random() > 0.99):
                print(i)
            newSubA = subA.copy()
            newSubA.H.remove_edges_from([c])
            if(c in seqConstraint.keys() and seqConstraint[c] in newSubA.H.edges()):
                pass # This means a feasability constraint has been failed!
            else:
                G.add_node(i, value=newSubA)
                G.add_edge(p,i, a=c, r=R(p,c))
                G = recurGen(i, newSubA, G)
            
            i = G.number_of_nodes() + 1
        return G
    else:
        return G

# Checking Feasability
def P(sN, s, a):
    # Geometric Feasability
    pass

    # Sequential Constraints again


def genRewards(assembled):
    Rewards = {i: random.randint(0, 10) for i in list(assembled.H.edges())}
    return Rewards

def R(s, a):
    return Rewards[s]

## Preliminary Code (Object Classes)

In [3]:
class part:
    def __init__(self, number):
        self.num = number
        self.length = 1
        self.width = 1
        self.pos = []
        self.vel = []
    
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    def __str__(self):
        return "Part " + str(self.num)
    def __repr__(self):
        return "Part " + str(self.num)

    
class subAssembly:
    def __init__(self):
        self.H = nx.Graph()
    
    def copy(self):
        new = subAssembly()
        new.H = self.H.copy()
        return new
    
    def getNCL(self):
        return str(len(list(self.H.edges())))
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    def __str__(self):
        return "NCL: " + self.getNCL()
    def __repr__(self):
        return "NCL: " + self.getNCL()

## Scenario Initialization

In [5]:
# Initialize the inputs (will later read in from file [JSON])
numAgents = 1
numParts = 4
numConnections = int(numParts*(numParts-1)/2)
numActions = int(sum([math.factorial(numConnections)/math.factorial(i) for i in range(0, numConnections)]))
numStates = numActions+1

#SEQUENCING CONSTRAINTS (will later read in from file [JSON])
# Of the format {connection touple: connection touple that must be remove first}

# seqConstraint = {}
seqConstraint = {(0, 2): (0, 1),
                 (1, 3): (2, 3),
                 (1, 2): (1, 3),
                 (0, 1): (0, 2)}

print("CONFIGURATION: \n\t%d Parts,\n\t%d Connections in Assembly,\n\t%d Potential Subassemblies, \n\t%d Potential Edges, \n\t%d Sequential Constraints"\
     % (numParts, numConnections, numStates, numActions, len(seqConstraint)))

CONFIGURATION: 
	4 Parts,
	6 Connections in Assembly,
	1957 Potential Subassemblies, 
	1956 Potential Edges, 
	4 Sequential Constraints


In [6]:
# Initialize the inputs (will later read in from file [JSON])
numAgents = 1
numParts = 5
numConnections = int(numParts*(numParts-1)/2)
numActions = int(sum([float(math.factorial(numConnections))/float(math.factorial(i)) for i in range(0, numConnections)]))
numStates = numActions+1

#SEQUENCING CONSTRAINTS (will later read in from file [JSON])
# Of the format {connection touple: connection touple that must be remove first}

seqConstraint = {}
# seqConstraint = {(0, 2): (0, 1),
#                  (1, 3): (2, 3),
#                  (1, 2): (1, 3),
#                  (0, 1): (0, 2)}

print("CONFIGURATION: \n\t%d Parts,\n\t%d Connections in Assembly,\n\t%d Subassemblies, \n\t%d Potential Edges, \n\t%d Sequential Constraints"\
     % (numParts, numConnections, numStates, numActions, len(seqConstraint)))

CONFIGURATION: 
	5 Parts,
	10 Connections in Assembly,
	9864101 Subassemblies, 
	9864100 Potential Edges, 
	0 Sequential Constraints


## Running the Assembly Generation

In [ ]:
# Simple initial initialization (will later read in from file [JSON])
Rewards = {i: random.randint(0, 10) for i in range(numStates)}

start = time.time()
assembled = subAssembly()
for i in range(numParts):
    n = part(i)
    assembled.H.add_node(i, value=n)
    assembled.H.add_edges_from([(i,j) for j in range(numParts) if i != j])
#Rewards = genRewards(assembled)

# Initialize the assembly graph
G = nx.DiGraph()
G.add_node(1, value=assembled)
G = recurGen(1, assembled, G)
end = time.time()
print("Assembly Generation Time: %f" % (end-start))
print(end-start)


print(list(assembled.H.edges()))
print("DONE: %d Nodes and %d Edges" % (G.number_of_nodes(), G.number_of_edges()))

290
443
619
622
657
672
717
1294
1471
1608
1622
1840
1868
2042
2139
2186
2187
2205
2247
2425
2603
2769
2971
2981
3029
3031
3040
3275
3404
3410
3451
3619
3630
3781
3885
3916
4049
4050
4253
4370
4755
4877
4983
5010
5378
5567
5576
5882
5893
6247
6263
6326
6406
6474
6537
6558
6651
6954
6996
7016
7017
7056
7163
7488
7624
7630
7718
7786
7827
8018
8234
8248
8362
8366
8479
8577
8588
8589
8688
8764
8778
8789
8970
9157
9227
9296
9301
9422
9465
9642
9851
10014
10243
10258
10267
10292
10488
10557
10563
10902
10957
11069
11172
11515
11656
11667
11706
11740
11896
11977
12031
12138
12258
12282
12607
12689
12713
12743
12804
13029
13085
13133
13154
13259
13381
13680
13740
14071
14215
14282
14284
14295
14430
14445
14463
14519
14521
14593
14609
14628
14669
14786
14933
14935
15050
15083
15166
15440
15527
15646
15755
16017
16272
16561
16727
16738
16797
16800
17002
17058
17060
17074
17121
17167
17206
17224
17243
17254
17467
17539
18305
18335
18451
18482
18739
18840
18895
18907
19160
19586
19674
19885
19951


143569
143653
143757
143800
143918
143939
144271
144278
144361
144431
144579
144676
144805
144901
144917
145020
145041
145133
145136
145204
145361
145371
145399
145635
145975
146174
146251
146368
146406
146407
146562
146819
147019
147111
147222
147253
147266
147386
147446
147531
147538
147815
147982
148001
148062
148182
148218
148297
148364
148378
148514
148570
148622
148700
148728
148792
148800
148818
148828
148891
149244
149286
149659
149665
149747
149812
149823
149910
150007
150049
150184
150194
150199
150298
150428
150437
150457
150679
150783
150877
150907
150980
150986
151032
151197
151253
151329
151648
151771
151859
152069
152140
152267
152338
152385
152526
152628
152675
152758
153021
153295
153307
153375
153624
153699
153858
153987
154159
154444
154480
154496
154647
154707
154770
154823
155018
155031
155042
155050
155143
155225
155403
155656
155855
155946
156020
156068
156181
156216
156424
156507
156571
156756
156873
156951
157355
157592
157731
157764
157845
157918
157920
158283

268125
268267
268333
268363
268577
268869
268893
269067
269268
269318
269331
269512
269717
270102
270191
270377
270399
270523
270742
270758
270900
271200
271241
271312
271363
271590
271769
271771
271929
271964
272066
272114
272139
272190
272229
272375
272437
272464
272513
272535
272916
273106
273112
273185
273236
273325
273388
273405
273478
273563
273757
273760
274094
274340
274423
274621
274719
275037
275157
275299
275421
275730
275780
275812
275873
275900
275937
276008
276045
276159
276284
276421
276439
276471
276475
276502
276516
276583
276653
276783
276963
276976
277021
277035
277052
277091
277144
277281
277384
277617
277621
277857
277963
278031
278353
278432
278457
278864
279027
279032
279073
279180
279229
279298
279326
279355
279377
279501
279581
279585
279993
280026
280116
280171
280201
280271
280328
280337
280376
280779
281052
281091
281113
281118
281147
281635
281644
281717
281762
282050
282104
282121
282132
282191
282243
282270
282478
282487
282688
282838
282882
283020
283088

398933
399024
399077
399157
399231
399583
399649
399652
399879
400114
400165
400286
400376
400474
400537
400554
400557
400665
401011
401198
401216
401244
401295
401376
401663
401723
401827
402011
402269
402315
402389
402419
402498
402584
402682
402697
402717
402821
402848
402871
402916
402930
403005
403147
403148
403156
403248
403415
403438
403539
403585
403714
403772
403776
403780
403788
403789
403792
403799
404049
404080
404200
404362
404793
404871
405150
405312
405338
405379
405523
405541
405666
405764
405927
405956
406062
406097
406221
406341
406379
406519
406582
406764
406824
406829
406850
406953
407247
407308
407388
407415
407685
407739
407893
408004
408072
408089
408113
408230
408267
408273
408391
408527
408751
408886
408953
408964
409040
409064
409089
409279
409304
409316
409420
409481
409497
409523
409566
409751
409804
409895
410002
410045
410133
410433
410521
410596
410618
410682
410825
410850
410985
411051
411189
411203
411216
411229
411247
411423
411714
411857
411904
411915

523185
523290
523390
523482
523993
524030
524093
524162
524331
524577
524583
524601
524710
524786
524969
525170
525206
525257
525382
525593
525721
525724
525745
525749
525843
525913
525987
526063
526090
526103
526111
526173
526177
526254
526529
526547
526736
526969
527106
527196
527276
527307
527317
527386
527543
527592
527701
527786
527866
528042
528128
528145
528212
528247
528352
528455
528665
528716
528769
528773
528877
529017
529062
529126
529199
529246
529517
529697
529750
529786
529789
530005
530087
530121
530257
530333
530336
530804
531134
531194
531329
531466
531574
531746
531782
531783
531788
531849
531889
531894
531950
531980
532038
532054
532062
532293
532434
532547
532563
532683
532693
533173
533295
533371
533380
533587
533633
533644
533699
533757
533932
533985
533998
534054
534137
534152
534283
534297
534374
534450
534849
534898
534957
535044
535045
535244
535254
535322
535425
535561
535880
535927
536237
536526
536663
536679
536908
536913
537118
537286
537364
537539
537773

...

6147629
6147693
6147801
6147883
6148213
6148374
6148495
6148651
6148702
6148757
6148774
6148860
6148947
6149003
6149154
6149195
6149316
6149344
6149418
6149429
6149466
6149479
6149879
6149882
6149883
6150047
6150087
6150183
6150252
6150262
6150334
6150438
6150554
6150621
6150672
6150703
6150734
6150829
6151173
6151400
6151452
6151513
6151766
6152016
6152024
6152045
6152093
6152114
6152118
6152602
6152628
6152798
6152881
6152997
6153020
6153235
6153286
6153401
6153525
6153594
6153621
6153652
6153728
6153752
6153785
6153836
6153873
6153978
6154235
6154268
6154298
6154310
6154385
6154441
6154492
6154584
6154689
6154710
6154859
6155115
6155193
6155291
6155344
6155403
6155444
6155586
6155745
6155799
6155975
6156036
6156210
6156297
6156358
6156386
6156453
6156598
6156726
6156754
6157019
6157076
6157195
6157270
6157342
6157422
6157431
6157516
6157609
6157688
6157859
6157906
6157951
6158014
6158226
6158454
6158519
6158573
6158715
6158776
6158981
6159031
6159109
6159182
6159244
6159322
6159530


6259546
6259567
6259603
6259672
6259701
6259912
6260003
6260071
6260178
6260407
6260417
6260440
6260468
6260521
6260550
6260765
6260801
6260807
6260890
6261111
6261281
6261318
6261434
6261584
6261708
6261735
6261832
6262123
6262172
6262220
6262225
6262293
6262410
6262420
6262591
6262620
6262791
6262862
6263266
6263368
6263587
6263666
6263730
6263820
6263887
6263966
6264036
6264130
6264246
6264336
6264359
6264373
6264404
6264425
6264548
6264611
6264671
6264696
6264850
6265067
6265145
6265202
6265322
6265329
6265333
6265431
6265565
6265568
6265956
6265978
6266120
6266482
6266676
6266684
6266951
6267128
6267165
6267250
6267305
6267314
6267550
6267569
6267753
6267811
6267869
6267921
6267954
6268078
6268102
6268239
6268254
6268281
6268304
6268417
6268529
6268536
6268659
6268818
6268845
6268892
6268930
6268958
6269016
6269114
6269137
6269143
6269168
6269180
6269238
6269350
6269404
6269505
6269605
6269612
6269695
6269721
6269789
6269849
6269861
6269982
6270039
6270371
6270552
6270633
6270670


6367196
6367327
6367526
6367719
6367723
6367853
6367969
6368050
6368414
6368519
6368545
6368622
6368651
6368810
6368917
6369044
6369073
6369079
6369098
6369135
6369137
6369144
6369164
6369221
6369269
6369643
6369827
6370272
6370519
6370578
6370642
6370745
6370751
6370886
6370980
6371041
6371098
6371241
6371308
6371461
6371690
6371926
6372012
6372304
6372330
6372350
6372369
6372379
6372653
6372671
6372712
6372989
6373105
6373165
6373204
6373298
6373477
6373479
6373639
6373676
6373687
6373748
6373855
6373950
6374014
6374031
6374057
6374253
6374258
6374377
6374449
6374639
6374809
6374872
6374906
6375014
6375162
6375200
6375267
6375349
6375351
6375369
6375419
6375459
6375480
6375708
6375828
6375878
6375929
6375951
6376029
6376227
6376400
6376542
6376851
6376877
6376980
6377083
6377430
6377545
6377785
6377862
6377977
6378015
6378021
6378040
6378322
6378623
6378677
6379141
6379276
6379309
6379364
6379517
6379560
6379667
6379772
6379845
6379921
6380017
6380236
6380291
6380438
6380482
6380556


6477210
6477735
6477839
6477883
6477914
6477936
6477953
6478085
6478104
6478113
6478201
6478356
6478598
6478693
6478697
6478704
6478746
6478862
6478934
6479014
6479228
6479317
6479326
6479344
6479429
6479450
6480051
6480098
6480124
6480174
6480300
6480355
6480403
6480442
6480477
6480644
6480689
6480825
6480846
6480913
6480929
6481112
6481215
6481245
6481345
6481433
6481649
6481862
6481866
6481874
6482068
6482368
6482457
6482924
6483003
6483099
6483295
6483327
6483363
6483441
6483506
6483562
6483597
6483785
6483871
6483883
6483935
6483981
6484032
6484102
6484114
6484226
6484329
6484334
6484482
6484567
6484590
6484669
6484726
6484766
6484799
6484811
6484837
6485198
6485226
6485490
6485491
6485516
6485609
6485630
6485710
6485829
6485889
6485937
6486127
6486224
6486264
6486644
6486839
6487003
6487121
6487184
6487370
6487411
6487447
6487600
6487647
6487687
6488083
6488113
6488131
6488146
6488251
6488313
6488326
6488552
6488734
6488876
6488979
6489152
6489198
6489286
6489477
6489654
6489711


6585760
6585764
6585827
6585833
6585862
6585893
6586279
6586432
6586660
6586937
6586964
6587007
6587124
6587139
6587155
6587569
6587806
6588053
6588089
6588201
6588319
6588336
6588379
6588396
6588457
6588492
6588545
6588551
6588605
6588740
6588768
6588816
6588896
6588913
6589362
6589445
6589647
6589715
6589845
6589962
6590045
6590054
6590056
6590091
6590133
6590136
6590176
6590340
6590516
6590524
6590606
6590998
6591021
6591091
6591143
6591220
6591346
6591374
6591436
6591718
6591775
6591801
6591874
6592096
6592305
6592515
6592561
6592569
6592646
6593474
6593479
6593512
6593539
6593802
6594038
6594075
6594158
6594170
6594233
6594489
6594552
6594569
6594581
6594646
6594674
6594677
6594752
6594808
6594865
6594886
6595109
6595242
6595323
6595344
6595429
6595717
6595842
6595894
6596164
6596252
6596347
6596502
6596589
6596715
6596858
6596977
6597030
6597043
6597069
6597182
6597227
6597260
6597295
6597336
6597517
6597597
6597616
6597691
6597787
6598004
6598056
6598178
6598188
6598256
6598368


## Value Iteration

In [ ]:
def ValueIteration(G, H, maxIter, eps):
    S = range(1, G.number_of_nodes()+1)
    A = list(H.edges())

    V = {s: 0.0 for s in S}
    Vnew = V.copy()
    π = {s: (0, (0,0)) for s in S}
    delta = max([abs(V[s] - Vnew[s]) for s in S])

    c = 0
    while(c <= maxIter):
        delta = 0
        for s in S:
            nextStates = G[s]
            for sN in nextStates.items():
                newV = sN[1]['r'] + V[sN[0]]
                if(newV > Vnew[s]):
                    Vnew[s] = newV
                    π[s] = (sN[0], sN[1]['a'])
        delta = max([abs(V[s] - Vnew[s]) for s in S])
        #print(c, delta)
        if(delta < eps):
            return Vnew, π
        V = Vnew.copy()
        c += 1
    print("Done")

maxIter = 100000
eps = 1e-6
start = time.time()
V, π = ValueIteration(G, assembled.H, maxIter, eps)
end = time.time()
print("Value Iteration Time: %f" % (end-start))
print(end-start)

# pp.pprint(V)
# pp.pprint(π)

# Graphing Suite

In [ ]:
try:
    pos = hierarchy_pos(G,1)
except:
    pos = nx.planar_layout(G)
actions = nx.get_edge_attributes(G,'r')
labels = nx.get_node_attributes(G,'r')
nx.draw(G, pos, with_labels=True, font_color="#FFFFFF")
nx.draw_networkx_edge_labels(G, pos, edge_labels = actions)
#nx.draw_networkx_labels(G, pos, labels)
#nx.draw(G, with_labels=True, font_color="#FFFFFF")


try:
    S = range(1, G.number_of_nodes()+1)
    path = [1, π[1][0]]
    path_edges = [(1, path[-1])]
    while(True):
        currState = path[-1]
        nextState = π[currState][0]
        if(nextState in S):
            path.append(nextState)
            path_edges.append((currState, nextState))
        else:
            break
    print(path_edges)
    nx.draw_networkx_nodes(G,pos,nodelist=path,node_color='r')
    nx.draw_networkx_edges(G,pos,edgelist=path_edges,edge_color='r',width=1)
except:
    print("No Policy Found")

print("DONE: %d Nodes and %d Edges" % (G.number_of_nodes(), G.number_of_edges()))

In [ ]:
nx.draw(assembled.H, with_labels=True, font_color="#FFFFFF")

In [ ]:
V = G.copy()
try:
    plotVis(V)
except:
    try:
        plotVis(V)
    except:
        print("Nope!")